In [1]:
# 2025-10-19 run it from "python 3" (ipykernel) on gl-base-env-j4-ces with 1 T4 gpu


# preparing data

## copy data from gcs bucket 
merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data

In [ ]:
# gsutil ls -r gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data/
# gsutil -m cp -r gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data/* /home/jupyter/mluser/git/cube/data/dataspace/polyhaven_tiny/

# (base) jupyter@gl-base-env-j4-ces:~/mluser/git/cube/data/dataspace/polyhaven_tiny$ ls -lh
# total 1.6G
# -rw-r--r-- 1 jupyter jupyter 1.5G Oct 22 00:03 cubediff_train.tar
# -rw-r--r-- 1 jupyter jupyter 111M Oct 22 00:02 cubediff_val.tar
# drwxr-xr-x 2 jupyter jupyter  36K Oct 22 00:02 latents
# drwxr-xr-x 6 jupyter jupyter 4.0K Oct 22 00:03 raw


# set up env

In [1]:
import os
os.environ["PYTHONPATH"] = "/home/jupyter/mluser/git/cube"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
# Set torch compile backend
os.environ["TORCH_COMPILE_BACKEND"] = "inductor"
os.environ["ACCELERATE_CONFIG_FILE"]="/home/jupyter/mluser/git/cube/cl/config/accelerate_config.yaml"
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"

## install packages

In [2]:
# before installation, check what installed
# (base) jupyter@gl-base-env-j4-ces:~/mluser/git/cube/data/dataspace/polyhaven_tiny$ pip list | grep "torch\|diff\|peft\|hug\|acc"
# torch                                    2.8.0
# torch-model-archiver                     0.12.0
# torchserve                               0.12.0

In [3]:
# 2015-10-21
# Install compatible versions without PEFT
!pip install diffusers==0.21.4 huggingface-hub==0.16.4 -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple

# Upgrade to versions that work together
# !pip install huggingface-hub==0.21.4 accelerate==1.0.1 -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple
!pip install accelerate==1.0.1 -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple

# 2025-10-21 Upgrade to latest compatible versions
# !pip install huggingface-hub==0.25.1 transformers==4.35.2 diffusers==0.25.1 -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple
!pip install huggingface-hub==0.25.1 transformers==4.35.2 diffusers==0.25.1 -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple

Looking in indexes: https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple
  Using cached https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/packages/packages/2e/ed/58a13f88dfcdd1bcfeabf44d4c9861979551339348e579fd6559c64b12e0/diffusers-0.21.4-py3-none-any.whl (1.5 MB)
  Using cached https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/packages/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.25.1
    Uninstalling huggingface-hub-0.25.1:
      Successfully uninstalled huggingface-hub-0.25.1
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.25.1
    Uninstalling diffusers-0.25.1:90m╺━━━━━━━━━━━━━━━━━━━ 1/2 [diffusers]
      Successfully uninstalled diffusers-0.25.1━━━━━━━━━━━━━━━ 1/2 [diffusers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# 2025-10-25, 2025-7-7 installed compatible diffusers huggingface_hub transformers to support torch.compile
!pip install torch==2.1.2 torchvision==0.16.2  \
     opencv-python==4.8.1.78 opencv-contrib-python==4.8.1.78 matplotlib==3.8.2 tqdm==4.66.1 einops==0.7.0 \
     xformers requests pillow openexr opencv-python-headless wandb \
     bitsandbytes webdataset timm accelerate deepspeed imageio \
     diffusers==0.25.1 huggingface_hub==0.25.1 transformers==4.35.2 \
     -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple

Looking in indexes: https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple


In [5]:
import OpenEXR

In [6]:
!pip show OpenEXR

Name: OpenEXR
Version: 3.4.2
Summary: Python bindings for the OpenEXR image file format
Home-page: https://openexr.com
Author: 
Author-email: Contributors to the OpenEXR project <info@openexr.com>
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy
Required-by: 


## check packages versions

In [7]:
!pip list | grep "diff\|hug\|acc\|trans\|torch\|peft"

accelerate                               1.0.1
diffusers                                0.25.1
huggingface-hub                          0.25.1
sentence-transformers                    5.1.2
torch                                    2.1.2
torch-model-archiver                     0.12.0
torchserve                               0.12.0
torchvision                              0.16.2
transformers                             4.35.2


# config shm

In [1]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             59G     0   59G   0% /dev
tmpfs            12G  768K   12G   1% /run
/dev/sda1       148G   45G   98G  32% /
tmpfs            59G     0   59G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
/dev/sda15      124M   11M  114M   9% /boot/efi
/dev/sdb        147G   39G  108G  27% /home/jupyter


In [2]:
# allocate more memory for shm
!sudo mount -o remount,size=64G /dev/shm

In [3]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             59G     0   59G   0% /dev
tmpfs            12G  768K   12G   1% /run
/dev/sda1       148G   45G   98G  32% /
tmpfs            64G     0   64G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
/dev/sda15      124M   11M  114M   9% /boot/efi
/dev/sdb        147G   39G  108G  27% /home/jupyter


# copied HF/hub and torch.hub from gcs to this VM

In [ ]:
# on target VM:
# mkdir -p ~/.cache/huggingface/hub
# gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/* \
#   ~/.cache/huggingface/hub/


In [23]:
# !mkdir -p ~/.cache/huggingface/hub

In [24]:
# !gsutil -m cp -r \
#    gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/* \
#    ~/.cache/huggingface/hub/


Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/version.txt...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/version_diffusers_cache.txt...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/.locks/models--openai--clip-vit-base-patch32/182766ce89b439768edadda342519f33802f5364.lock...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/.locks/models--openai--clip-vit-base-patch32/4fdaf6842dd5a725b940c92b6b692490ce59d548.lock...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/.locks/models--openai--clip-vit-base-patch32/564c0ebd5ce29c4ee4864004aee693deadd3128c.lock...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/.locks/models--runwayml--stable-diffusion-v1-5/19da7aaa4b880e59d56843f1fcb4dd9b599c28a1d9d9af7c1143057c8ffae9f1.lock...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/c

In [4]:
!ls -lh ~/.cache/huggingface/hub

total 20K
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--openai--clip-vit-base-patch32
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--openai--clip-vit-large-patch14
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--runwayml--stable-diffusion-v1-5
-rw-r--r-- 1 jupyter jupyter    1 Oct 25 06:19 version.txt
-rw-r--r-- 1 jupyter jupyter    1 Oct 25 06:19 version_diffusers_cache.txt


In [1]:
!rm -rf ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32

In [4]:
# on target VM:
!gsutil -m cp -r gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/* \
  ~/.cache/huggingface/hub/


Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/blobs/182766ce89b439768edadda342519f33802f5364...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/blobs/4fdaf6842dd5a725b940c92b6b692490ce59d548...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/blobs/564c0ebd5ce29c4ee4864004aee693deadd3128c...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/blobs/5a12a1eb250987a4eee0e3e7d7338c4b22724be1...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/blobs/99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/

In [5]:
!ls -lh ~/.cache/huggingface/hub

total 20K
drwxr-xr-x 5 jupyter jupyter 4.0K Oct 29 14:14 models--openai--clip-vit-base-patch32
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--openai--clip-vit-large-patch14
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--runwayml--stable-diffusion-v1-5
-rw-r--r-- 1 jupyter jupyter    1 Oct 25 06:19 version.txt
-rw-r--r-- 1 jupyter jupyter    1 Oct 25 06:19 version_diffusers_cache.txt


In [ ]:
# on target VM:
# mkdir -p ~/.cache/torch/hub
# gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/* \
#   ~/.cache/torch/hub/


In [26]:
# !mkdir -p ~/.cache/torch/hub

In [28]:
# !gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/* \
#   ~/.cache/torch/hub/


Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/checkpoints/vgg16-397923af.pth...
\ [1/1 files][527.8 MiB/527.8 MiB] 100% Done                                    
Operation completed over 1 objects/527.8 MiB.                                    


In [5]:
!ls -lh ~/.cache/torch/hub/

total 4.0K
drwxr-xr-x 2 jupyter jupyter 4.0K Oct 25 06:27 checkpoints


In [6]:
!ls -lh ~/.cache/torch/hub/checkpoints

total 528M
-rw-r--r-- 1 jupyter jupyter 528M Oct 25 06:27 vgg16-397923af.pth


# check models

In [6]:
!ls -lh ~/.cache/huggingface/hub/

total 20K
drwxr-xr-x 5 jupyter jupyter 4.0K Oct 29 14:14 models--openai--clip-vit-base-patch32
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--openai--clip-vit-large-patch14
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--runwayml--stable-diffusion-v1-5
-rw-r--r-- 1 jupyter jupyter    1 Oct 25 06:19 version.txt
-rw-r--r-- 1 jupyter jupyter    1 Oct 25 06:19 version_diffusers_cache.txt


In [7]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/:
total 12K
drwxr-xr-x 2 jupyter jupyter 4.0K Oct 29 14:14 blobs
drwxr-xr-x 3 jupyter jupyter 4.0K Oct 29 14:14 refs
drwxr-xr-x 4 jupyter jupyter 4.0K Oct 29 14:14 snapshots

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs:
total 1.2G
-rw-r--r-- 1 jupyter jupyter 843K Oct 29 14:14 182766ce89b439768edadda342519f33802f5364
-rw-r--r-- 1 jupyter jupyter  592 Oct 29 14:14 4fdaf6842dd5a725b940c92b6b692490ce59d548
-rw-r--r-- 1 jupyter jupyter 2.2M Oct 29 14:14 564c0ebd5ce29c4ee4864004aee693deadd3128c
-rw-r--r-- 1 jupyter jupyter  316 Oct 29 14:14 5a12a1eb250987a4eee0e3e7d7338c4b22724be1
-rw-r--r-- 1 jupyter jupyter 578M Oct 29 14:14 99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d
-rw-r--r-- 1 jupyter jupyter  389 Oct 29 14:14 9bfb42aa97dcd61e89f279ccaee988bccb4fabae
-rw-r--r-- 1 jupyter jupyter 4.1K Oct 29 14:14 a2a88b96561196777ca173b15309ea859f4d2ce0
-rw-r--r-- 1 jupy

In [8]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs 

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs:
total 1.2G
-rw-r--r-- 1 jupyter jupyter 843K Oct 29 14:14 182766ce89b439768edadda342519f33802f5364
-rw-r--r-- 1 jupyter jupyter  592 Oct 29 14:14 4fdaf6842dd5a725b940c92b6b692490ce59d548
-rw-r--r-- 1 jupyter jupyter 2.2M Oct 29 14:14 564c0ebd5ce29c4ee4864004aee693deadd3128c
-rw-r--r-- 1 jupyter jupyter  316 Oct 29 14:14 5a12a1eb250987a4eee0e3e7d7338c4b22724be1
-rw-r--r-- 1 jupyter jupyter 578M Oct 29 14:14 99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d
-rw-r--r-- 1 jupyter jupyter  389 Oct 29 14:14 9bfb42aa97dcd61e89f279ccaee988bccb4fabae
-rw-r--r-- 1 jupyter jupyter 4.1K Oct 29 14:14 a2a88b96561196777ca173b15309ea859f4d2ce0
-rw-r--r-- 1 jupyter jupyter 578M Oct 29 14:14 a63082132ba4f97a80bea76823f544493bffa8082296d62d71581a4feff1576f
-rw-r--r-- 1 jupyter jupyter 513K Oct 29 14:14 bbfec752c9a675946c6dce106def6f35c882dcc2


In [9]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/182766ce89b439768edadda342519f33802f5364

-rw-r--r-- 1 jupyter jupyter 843K Oct 29 14:14 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/182766ce89b439768edadda342519f33802f5364


In [10]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/4fdaf6842dd5a725b940c92b6b692490ce59d548

-rw-r--r-- 1 jupyter jupyter 592 Oct 29 14:14 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/4fdaf6842dd5a725b940c92b6b692490ce59d548


In [11]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/564c0ebd5ce29c4ee4864004aee693deadd3128c

-rw-r--r-- 1 jupyter jupyter 2.2M Oct 29 14:14 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/564c0ebd5ce29c4ee4864004aee693deadd3128c


In [12]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/9bfb42aa97dcd61e89f279ccaee988bccb4fabae

-rw-r--r-- 1 jupyter jupyter 389 Oct 29 14:14 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/9bfb42aa97dcd61e89f279ccaee988bccb4fabae


In [13]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/a2a88b96561196777ca173b15309ea859f4d2ce0

-rw-r--r-- 1 jupyter jupyter 4.1K Oct 29 14:14 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/a2a88b96561196777ca173b15309ea859f4d2ce0


In [14]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/bbfec752c9a675946c6dce106def6f35c882dcc2

-rw-r--r-- 1 jupyter jupyter 513K Oct 29 14:14 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/bbfec752c9a675946c6dce106def6f35c882dcc2


In [15]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/

total 8.0K
drwxr-xr-x 2 jupyter jupyter 4.0K Oct 29 14:14 3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268
drwxr-xr-x 2 jupyter jupyter 4.0K Oct 29 14:14 c237dc49a33fc61debc9276459120b7eac67e7ef


In [16]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268

total 581M
-rw-r--r-- 1 jupyter jupyter 4.1K Oct 29 14:14 config.json
-rw-r--r-- 1 jupyter jupyter 513K Oct 29 14:14 merges.txt
-rw-r--r-- 1 jupyter jupyter  316 Oct 29 14:14 preprocessor_config.json
-rw-r--r-- 1 jupyter jupyter 578M Oct 29 14:14 pytorch_model.bin
-rw-r--r-- 1 jupyter jupyter  389 Oct 29 14:14 special_tokens_map.json
-rw-r--r-- 1 jupyter jupyter 2.2M Oct 29 14:14 tokenizer.json
-rw-r--r-- 1 jupyter jupyter  592 Oct 29 14:14 tokenizer_config.json
-rw-r--r-- 1 jupyter jupyter 843K Oct 29 14:14 vocab.json


In [17]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/182766ce89b439768edadda342519f33802f5364

ls: cannot access '/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/182766ce89b439768edadda342519f33802f5364': No such file or directory


In [18]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/refs/

total 8.0K
-rw-r--r-- 1 jupyter jupyter   40 Oct 29 14:14 main
drwxr-xr-x 3 jupyter jupyter 4.0K Oct 29 14:14 refs


In [19]:
!ls -lh ~/.cache/torch/hub/checkpoints

total 528M
-rw-r--r-- 1 jupyter jupyter 528M Oct 25 06:27 vgg16-397923af.pth


## test they can be loaded from local cache

In [19]:
!ls -lh  ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268


total 3.5M
-rw-r--r-- 1 jupyter jupyter 4.1K Oct 25 06:19 config.json
-rw-r--r-- 1 jupyter jupyter 513K Oct 25 06:19 merges.txt
-rw-r--r-- 1 jupyter jupyter  389 Oct 25 06:19 special_tokens_map.json
-rw-r--r-- 1 jupyter jupyter 2.2M Oct 25 06:19 tokenizer.json
-rw-r--r-- 1 jupyter jupyter  592 Oct 25 06:19 tokenizer_config.json
-rw-r--r-- 1 jupyter jupyter 843K Oct 25 06:19 vocab.json


In [16]:
!pip list | grep "torch"

torch                                    2.1.2
torch-model-archiver                     0.12.0
torchserve                               0.12.0
torchvision                              0.16.2


In [6]:
from transformers import CLIPModel, CLIPProcessor

# This should now work without network calls
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=True, local_files_only=False) # use_safetensors=True, caused the error of no safetensors for the downloaded model.
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", local_files_only=True)

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=False, local_files_only=True) # worked for the model downloaded on 2025-10-29 in "bbyus-trn-amlo-yieldoptimz-p01"
                                                                                                                # details at mluser/git/cube/training/notebooks/v2/download_copy_vit.ipynb
                                                                                                                # but the same vit model could be loaded well with "use_safetensors=True" 
                                                                                                                # in "bbyus-trn-amlo-yieldoptimz-p01"
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", local_files_only=True)

In [7]:
model.name_or_path

'openai/clip-vit-base-patch32'